In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/Tritangent_plane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
var('nv')
K1.<gamma> = NumberField(nv^4-4*nv^2+16)

In [3]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(K1)

In [4]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [5]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [6]:
line = Line([y, z])

In [7]:
general_cubic = Cubic(cubic_new, line, sing_cubics)

In [8]:
SE9 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({e : (-1/2*gamma^2 + 1)*c})

In [9]:
%time adm_SE9 = SE9.find_admissible_projectivities()

CPU times: user 9.44 s, sys: 275 ms, total: 9.72 s
Wall time: 1min 7s


In [10]:
len(adm_SE9)

1296

In [11]:
%time simm_SE9 = SE9.find_simmetries(adm_SE9)

CPU times: user 5.79 s, sys: 210 ms, total: 6 s
Wall time: 7.72 s


In [12]:
len(simm_SE9)

54

In [13]:
%time conds = find_conditions_for_subfamilies(SE9, adm_SE9, simm_SE9)

CPU times: user 4min 4s, sys: 5.21 s, total: 4min 9s
Wall time: 4min 15s


In [14]:
conds

[[13*c + (-gamma^3 - gamma^2 + 3*gamma + 3)*f],
 [4*c + (-gamma^3 + 2*gamma^2 - 4)*f],
 [4*c + (gamma^3 + 2*gamma^2 - 4)*f],
 [52*c + (gamma^3 - 4*gamma^2 + 12*gamma + 4)*f],
 [52*c + (-gamma^3 - 4*gamma^2 - 12*gamma + 4)*f],
 [13*c + (gamma^3 - gamma^2 - 3*gamma + 3)*f]]

In [15]:
sol = solve_linear_system(conds[1], [c], [f])

In [16]:
SE9_1 = SE9.subs({c:sol[0], f:sol[1]})

In [17]:
SE9_1

(-16*gamma^2 + 32)*x^2*y + (16*gamma^2 - 32)*x*y^2 + (16*gamma^2 - 32*gamma)*x^2*z + (-8*gamma^3 - 8*gamma^2 + 48*gamma - 32)*x*y*z + (8*gamma^2 - 16*gamma)*x*z^2 + (-8*gamma^3 + 16*gamma^2 + 32*gamma - 64)*x*y*t + (-8*gamma^2 - 16*gamma + 32)*y^2*t + (8*gamma^3 - 8*gamma^2 - 16*gamma + 32)*y*z*t + (8*gamma^3 - 16*gamma^2 + 32)*y*t^2

In [18]:
len(SE9_1.eckardt_points)

9

In [19]:
%time adm_SE9_1 = SE9_1.find_admissible_projectivities()

CPU times: user 4.98 s, sys: 246 ms, total: 5.22 s
Wall time: 40.1 s


In [20]:
len(adm_SE9_1)

1296

In [21]:
simm_SE9_1 = SE9_1.find_simmetries(adm_SE9_1)

In [22]:
len(simm_SE9_1)

108

In [24]:
perms = []
for el in simm_SE9_1:
    perms.append(Permutation(apply_proj_to_eck(el, SE9_1.eckardt_points)).to_cycles())

In [25]:
PermutationGroup(perms).order()

36

In [26]:
PermutationGroup(perms).structure_description()

'(C3 x C3) : C4'

In [28]:
adm_perm = SE9_1.find_admissible_permutations()
keys = list(SE9_1.cl_lines.keys())

In [29]:
perms_lines = []
for simm in simm_SE9_1:
    perm = adm_perm[adm_SE9_1.index(simm)]
    perms_lines.append(Permutation([perm.index(label)+1 for label in keys]).to_cycles())

In [30]:
Gr = PermutationGroup(perms_lines)

In [31]:
Gr.order()

108

In [32]:
Gr.structure_description()

'((C3 x C3) : C3) : C4'